In [133]:
import pandas as pd # type: ignore
import os
import glob
from pathlib import Path
import json
import re

### Define file paths and load data

In [142]:
# Specify the folder paths
olympics_folder_path = '../datasets/olympics/'
countries_folder_path='../datasets/countries/countries of the world.csv'

olympics_dataframes,countries_dataframes = [], []
# Get a list of all CSV files in the folder
oly_csv_files = glob.glob(os.path.join(olympics_folder_path, '*.csv')) #for olypics files
#country_csv_file = glob.glob(os.path.join(countries_folder_path, '*.csv')) #for country files

# Read each CSV file into a dataframe and store them in a list

for csv_file in oly_csv_files:
    df = pd.read_csv(csv_file)
    olympics_dataframes.append(df)

#combined dataframe with  all olympics data
olympics_combined_df = pd.concat(olympics_dataframes, ignore_index=True)

#display first 20 values of the combined dataframe
print(olympics_combined_df.head(20))



    NOC  Gold  Silver  Bronze  Total
0   GER    11      12       6     29
1   USA     9       9       7     25
2   AUT     9       7       7     23
3   RUS     8       6       8     22
4   CAN     7      10       7     24
5   SWE     7       2       5     14
6   KOR     6       3       2     11
7   SUI     5       4       5     14
8   ITA     5       0       6     11
9   FRA     3       2       4      9
10  NED     3       2       4      9
11  EST     3       0       0      3
12  NOR     2       8       9     19
13  CHN     2       4       5     11
14  CZE     1       2       1      4
15  CRO     1       2       0      3
16  AUS     1       0       1      2
17  JPN     1       0       0      1
18  FIN     0       6       3      9
19  POL     0       1       1      2


#### Basic Transformations fo Olympics

In [135]:
# Define a schema: ensuring column names and data types are consistent
olympics_combined_df.columns = [col.strip().lower().replace(' ', '_') for col in olympics_combined_df.columns]

# Display the updated schema
olympics_combined_df.dtypes

#saving the output
olympics_combined_df.to_csv('../output_files/combined_olympics.csv', index=False)


#### Load Countries data

In [136]:

#for countries
countries_df = pd.read_csv(countries_folder_path)
countries_df.head(10)


,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN
5,Angola,SUB-SAHARAN AFRICA,12127071,1246700,"9,7","0,13",0,"191,19",1900.0,"42,0","7,8","2,41","0,24","97,35",NaN,"45,11","24,2","0,096","0,658","0,246"
6,Anguilla,LATIN AMER. & CARIB,13477,102,"132,1","59,80","10,76","21,03",8600.0,"95,0","460,0",0,0,100,2,"14,17","5,34","0,04","0,18","0,78"
7,Antigua & Barbuda,LATIN AMER. & CARIB,69108,443,"156,0","34,54","-6,15","19,46",11000.0,"89,0","549,9","18,18","4,55","77,27",2,"16,93","5,37","0,038","0,22","0,743"
8,Argentina,LATIN AMER. & CARIB,39921833,2766890,"14,4","0,18","0,61","15,18",11200.0,"97,1","220,4","12,31","0,48","87,21",3,"16,73","7,55","0,095","0,358","0,547"
9,Armenia,C.W. OF IND. STATES,2976372,29800,"99,9","0,00","-6,47","23,28",3500.0,"98,6","195,7","17,55","2,3","80,15",4,"12,07","8,23","0,239","0,343","0,418"


### Column Preprocessing

In [137]:
# Ensuring column names and data types are consistent with that of Olympics dataset
countries_df.columns = [col.strip().lower().replace(' ', '_') for col in countries_df.columns]
print(countries_df.columns)
# Converting some datatypes

countries_df['population'] = countries_df['population'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
countries_df['gdp_($_per_capita)'] = countries_df['gdp_($_per_capita)'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

#method to make sure column names are consistent to avoid any issues while JOINs
def format_country_name(name):
    # Convert to lowercase, remove special characters, and trim whitespace using REGEX
    if pd.notna(name):
        name = re.sub(r'[^a-zA-Z\s]', '', name).strip().lower() 
    return name

# Display the updated schema
countries_df.dtypes


Index(['country', 'region', 'population', 'area_(sq._mi.)',
       'pop._density_(per_sq._mi.)', 'coastline_(coast/area_ratio)',
       'net_migration', 'infant_mortality_(per_1000_births)',
       'gdp_($_per_capita)', 'literacy_(%)', 'phones_(per_1000)', 'arable_(%)',
       'crops_(%)', 'other_(%)', 'climate', 'birthrate', 'deathrate',
       'agriculture', 'industry', 'service'],
      dtype='object')


country                                object
region                                 object
population                              int64
area_(sq._mi.)                          int64
pop._density_(per_sq._mi.)             object
coastline_(coast/area_ratio)           object
net_migration                          object
infant_mortality_(per_1000_births)     object
gdp_($_per_capita)                    float64
literacy_(%)                           object
phones_(per_1000)                      object
arable_(%)                             object
crops_(%)                              object
other_(%)                              object
climate                                object
birthrate                              object
deathrate                              object
agriculture                            object
industry                               object
service                                object
dtype: object

### Basic Transformations for Countries

In [138]:

# Handle missing values or duplicates
countries_df.dropna(inplace=True)
countries_df.drop_duplicates(inplace=True)

# Add row number to add primary key index to COuntries dataset
countries_df['country_id'] = countries_df.reset_index().index + 1
#reorder the postion of the country_id
countries_df = countries_df[['country_id'] + [col for col in countries_df.columns if col != 'country_id']]
#saving the output
countries_df.to_csv('../output_files/countries_data.csv', index=False)


## Combined Transformations

### 1. Normalized Data

In [139]:


################### 1. NORMALIZATIONS  #######################
#Add an artificial key to combine Olympics and Country data
# Adding a dictionary under datasets that maps each NOC to a country so we can get an accurate mapping
with open('../datasets/noc_countries.json', 'r') as f:
    country_mapping = json.load(f)

#country_name is the foreign key from Countries dataset that will be used in JOINs
olympics_combined_df['country'] = olympics_combined_df['noc'].map(country_mapping)



olympics_combined_df.to_csv('../output_files/olympics_normalized.csv', index=False)
countries_df.to_csv('../output_files/countries_normalized.csv',index=False)


###############################################

### 2.Denormized Data

In [140]:
##################### DENORMALIZATION ###############
# we need to combine the data from the 2 tables into 1 big table to avoid the use of JOINS
# this dataframe gives us the aggregated medals won
olympics_agg_df = olympics_combined_df.groupby('country').agg({
    'gold': 'sum',
    'silver': 'sum',
    'bronze': 'sum',
    'total':'sum'
}).reset_index()


# Since all the countires are unique and we have high cardinality it is a 1-1 mapping from the aggregated dataframe 
# to the countries dataframe

olympics_agg_df['country'] = olympics_agg_df['country'].apply(format_country_name)
countries_df['country'] = countries_df['country'].apply(format_country_name)
merged_df = pd.merge(olympics_agg_df, countries_df, on='country', how='right') #type of join can be inner or right depending on the end goal
print(merged_df.head(10))

merged_df.to_csv('../output_files/denormalized_data.csv',index=False)


            country  gold  silver  bronze  total  country_id  \
0       afghanistan   0.0     0.0     1.0    1.0           1   
1           albania   0.0     0.0     2.0    2.0           2   
2           algeria   6.0     3.0     5.0   14.0           3   
3          anguilla   NaN     NaN     NaN    NaN           4   
4  antigua  barbuda   NaN     NaN     NaN    NaN           5   
5         argentina   7.0     8.0    12.0   27.0           6   
6           armenia   2.0    10.0     5.0   17.0           7   
7             aruba   NaN     NaN     NaN    NaN           8   
8         australia  99.0   109.0   123.0  331.0           9   
9           austria  44.0    50.0    63.0  157.0          10   

                                region  population  area_(sq._mi.)  \
0        ASIA (EX. NEAR EAST)             31056997          647500   
1  EASTERN EUROPE                          3581655           28748   
2  NORTHERN AFRICA                        32930091         2381740   
3              